Developed By Hassan Murtaza 
Email: i170368@nu.edu.pk

libraries
- I use spacy as it has taught in the class
- I use regular expression for data cleansing
- Algorithms are implemented by myself(not use any library for that
- I use pandas for some basic operations
- I use confusion metrics for model evaluation

First of ALL I build 4 files from given zip and upload these 4 files into local of google colab

Table of content


Note: first 8 steps are with stopwords in text & last 4 steps without stopwords in text
1.   Load datasets into local of colab 
2.   Load datasets into pandas data frame from local csv
3.   Data Cleansing
4.   Building Documents for model trainings
5.   Generating Vocabulary
6.   Implementing Helper functions for naive bayes
7.   Model Training(NB and Boolean NB)
8.   Model Testing
9.   Evaluating Models(NB and Bo0lean NB)
10.  Stopword Removal
11.  Model Building(NB and Boolean NB with stopword removal)
12.  Model Testing
13.  Model Evaluation
14.  Conclusion




1. Load datasets into local of colab

In [3]:
from google.colab import files

select = files.upload()

Saving stopwords-ur.txt to stopwords-ur.txt
Saving Test_Fake.csv to Test_Fake.csv
Saving Test_Real.csv to Test_Real.csv
Saving Train_Fake.csv to Train_Fake.csv
Saving Train_Real.csv to Train_Real.csv


2. Load datasets into pandas data frame from local csv

    Reading csv into Pandas Data Frames

    1- Read train (real and fake) csv into pandas, then convert it into single docx

    2- Read test (real and fake) csv into pandas

In [54]:
import pandas as pd
import spacy

unlp = spacy.blank('ur')


stopwords = pd.read_csv("stopwords-ur.txt",header=None)
trainReal = pd.read_csv("Train_Real.csv")
trainFake = pd.read_csv("Train_Fake.csv")
# trainRealList = trainReal.drop('labels', inplace=False, axis=1).values.tolist()
# trainFakeList = trainFake.drop('labels', inplace=False, axis=1).values.tolist()
testReal = pd.read_csv("Test_Real.csv")
testFake = pd.read_csv("Test_Fake.csv")
testData = pd.concat([testReal, testFake])
testData = testData.sample(frac=1)
test_X =testData.drop('labels',axis=1,inplace=False).values.tolist() 
test_Y =testData.drop('features',axis=1,inplace=False).values.tolist()


**3. Cleaning Dataset**

1- The training data contain Numbers and alphabet, I removed them using regular expression


In [55]:
import re
def data_cleaning(sent):
  punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  sentCleaned = ""
  for char in sent:
    if char not in punctuations:
        sentCleaned = sentCleaned + char
  urduHaroof = "اآب پ ت  ث ٹ ج چ ح خ د ڈ ذ  رڑزژس ش ص ض ط ظ ع غ ف ق ک گ  ل م ن ں و ہ ھ ی ےٿ ڐ ؤئء"
  sentCleaned=re.sub(r'[0-9]', '', sentCleaned)
  sentCleaned=re.sub(r'[a-zA-Z]', '', sentCleaned)
  return sentCleaned

cleanedR = []
cleanedF = []
cleanedT = []

for i in trainReal['features']:
  cleanedR.append(data_cleaning(str(i)))

for i in trainFake['features']:
  cleanedF.append(data_cleaning(str(i)))

for i in testData['features']:
  cleanedT.append(data_cleaning(str(i)))


print("Train set and test sets are cleaned successfully")

Train set and test sets are cleaned successfully


**4. Generating documents for classes**

It means, divide the dataset into real examples and fake examples

In [56]:
docxReal = (" ".join(map(str,cleanedR))).split()
docxFake = (" ".join(map(str,cleanedF))).split()

print("Size of real doc", len(docxReal))
print("Size of Fake doc", len(docxFake))

Size of real doc 123774
Size of Fake doc 64543


**5. Generating Vocabulary**

This is the most important part where I build vocabulary

In [57]:
v1 = (" ".join(map(str,cleanedR))).split()
v2 = (" ".join(map(str,cleanedF))).split()
vocab1 = list(set(v1))
vocab2 = list(set(v2))
# It cancatenate 2 vcabs into single and then remove the redundant words
vocabulary = list(set(vocab1 + vocab2))

print("Vocab Length = ", len(vocabulary))


Vocab Length =  14256


**6. Helper Functions**

This section contains helper functions which are necessary for Naive Bayes

In [81]:
# It returns the count of a word in particular docx
def count_word(word,docx):
  return len(re.findall(word, str(docx)))

print((count_word('انہیں', docxReal)+1) / (len(docxReal) + len(vocabulary)))
# It returns the prior class probability
def find_prior(nb):
    if nb == 'real':
      prob = len(cleanedR) / (len(cleanedF) + len(cleanedR))
      return prob 
    elif nb =='fake':
      prob = len(cleanedF) / (len(cleanedF) + len(cleanedR))
      return prob



# This Funtion calculate the conditional probability
def conditional_probability(docx, vocabulary):
  probList = []
  totalsize = len(docx) + len(vocabulary)
  print(totalsize)
  for i in vocabulary:
    probList.append([i , (count_word(i, docx)+1) / (len(docxReal) + len(vocabulary))])
  return probList


# This is a helper function for test method
def probability(word, model):
  prob = 0.0003912193001521408
  for i in model:
    if i[0] == word:
      prob = i[1] 
  return prob


# This is a train function for traing the data
def train(classs, vocab):
  return conditional_probability(classs, vocab)

# This is test method for test the models on test data
import numpy as np
def test(Real, Fake, testData):
  import random
  result = list()
  for sent in testData:
    pR = []
    pF = []
    for word in sent:
      #print(" Real ", word,probability(word, Real))
      #print(" Fake ", word,probability(word, Fake))
      
       pR.append(probability(word, Real))
       pF.append(probability(word, Real))
    priorR = np.log(find_prior("real"))
    priorF = np.log(find_prior("fake"))

    pr = np.sum(np.log(pR)) + priorR
    pf = np.sum(np.log(pF)) + priorF


    if np.argmax([pr,pf]) == 0:
      result.append('Real')
    elif np.argmaz([pr,pf]) == 1:
       result.append("Fake")
  return result

# It removes the duplicates words
def unique_list(l):
    ulist = []
    [ulist.append(x) for x in l if x not in ulist]
    return ulist


# It will be used for removing stopwords
def remove_stopwords(word):
  for i in stopwords[stopwords.columns[0]]:
    if word == i:
      return True
  return False

0.0008983554299789901


7. Naive Bayes Implemention (without removing stopwords)

- Here, first I train model on both classes
- I also use laplace smoothing

In [60]:
realClassModel = train(docxReal, vocabulary)
fakeClassModel = train(docxFake, vocabulary)

138030
78799


Boolean Naive Bayes Implemention (without removing stopwords)

In [69]:
# First of all removing redundant words from each review 
trainRSetBoolean = list()
trainFSetBoolean = list()
testSetBoolean = list()

for i in cleanedR:
  trainRSetBoolean.append(' '.join(unique_list(str(i).split())))

for i in cleanedF:
  trainFSetBoolean.append(' '.join(unique_list(str(i).split())))

for i in cleanedT:
  testSetBoolean.append(' '.join(unique_list(str(i).split())))

# Now generating documents
docxBReal = (" ".join(map(str,trainRSetBoolean))).split()
docxBFake = (" ".join(map(str,trainFSetBoolean))).split()


print("Total words contain docxBReal==", len(docxBReal))
print("Total words contain docxBFake==", len(docxBFake))
# Boolean Naive Bayes Training
boolRModel = train(docxBReal, vocabulary)
boolFModel = train(docxBFake, vocabulary)

Total words contain docxBReal== 59994
Total words contain docxBFake== 33634
74250
47890


**8.Model Testing**

Here I am Testing my model

In [82]:
testNB = list()
testBNB = list()
for i in cleanedT:
  testNB.append(str(i).split())
for i in testSetBoolean:
  testBNB.append(str(i).split())

predNB = test(Real=realClassModel, Fake=fakeClassModel, testData=testNB)
predBNB = test(Real=boolRModel, Fake=boolFModel, testData=testBNB)

**9. Model Evaluation**

Confusion Matrix

# New section

In [83]:
from sklearn import metrics

print("Simple Naive Bayes Statistics Model Evaluation")
print(metrics.confusion_matrix(test_Y, predNB, labels=["Real","Fake"]))
print(metrics.classification_report(test_Y, predNB, labels=["Real","Fake"]))

print("\n\n")

print("Boolean Naive Bayes Statistics Model Evaluation")
print(metrics.confusion_matrix(test_Y, predBNB, labels=["Real","Fake"]))
print(metrics.classification_report(test_Y, predBNB, labels=["Real","Fake"]))

Simple Naive Bayes Statistics Model Evaluation
[[150   0]
 [ 90   0]]
              precision    recall  f1-score   support

        Real       0.62      1.00      0.77       150
        Fake       0.00      0.00      0.00        90

    accuracy                           0.62       240
   macro avg       0.31      0.50      0.38       240
weighted avg       0.39      0.62      0.48       240




Boolean Naive Bayes Statistics Model Evaluation
[[150   0]
 [ 90   0]]
              precision    recall  f1-score   support

        Real       0.62      1.00      0.77       150
        Fake       0.00      0.00      0.00        90

    accuracy                           0.62       240
   macro avg       0.31      0.50      0.38       240
weighted avg       0.39      0.62      0.48       240



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


10. Stopword Removal

    - Preparing Train Set for NB and Boolean NB

In [70]:
docxSWRealNB = []
docxSWFakeNB = []
docxSWRealBNB = []
docxSWFakeBNB = []
testSet = []


for i in docxReal:
  if remove_stopwords(i):
    continue
  docxSWRealNB.append(i)
print("Length of docx with real reviews for Niave Bayes", len(docxSWRealNB))

for i in docxFake:
  if remove_stopwords(i):
    continue
  docxSWFakeNB.append(i)
print("Length of docx with Fake reviews for Niave Bayes", len(docxSWFakeNB))


for i in docxBReal:
  if remove_stopwords(i):
    continue
  docxSWRealBNB.append(i)
print("Length of docx with real reviews for Boolean Niave Bayes", len(docxSWRealBNB))

for i in docxBFake:
  if remove_stopwords(i):
    continue
  docxSWFakeBNB.append(i)
print("Length of docx with Fake reviews for Boolean Niave Bayes", len(docxSWFakeBNB))


Length of docx with real reviews for Niave Bayes 96871
Length of docx with Fake reviews for Niave Bayes 49989
Length of docx with real reviews for Boolean Niave Bayes 52464
Length of docx with Fake reviews for Boolean Niave Bayes 29162



  - Preparing Test set for NB and Boolean NB 

In [71]:
testSWNB = []
testSWBNB = []

for i in cleanedT:
  sent = i.split()
  row = []
  for j in sent:
    if remove_stopwords(j):
      continue
    row.append(j)
  testSWNB.append(row)


for i in testSetBoolean:
  
  sent = i.split()
  row = []
  for j in sent:
    if remove_stopwords(j):
      continue
    row.append(j)
  testSWBNB.append(row)


11. Model Training (NB and Boolean NB)

In [72]:
nbSWRModel = train(docxSWRealNB, vocabulary)
nbSWFModel = train(docxSWFakeNB, vocabulary)

bnbSWRModel = train(docxSWRealBNB, vocabulary)
bnbSWFModel = train(docxSWFakeBNB, vocabulary)

111127
64245
66720
43418


**12. Model Testing**

In [84]:
predSWNB = test(Real=nbSWRModel, Fake=nbSWFModel, testData=testSWNB)
predSWBNB = test(Real=bnbSWRModel, Fake=bnbSWFModel, testData=testSWBNB)

13. Model Evaluation

In [85]:
print("Removed Stopwords: Simple Naive Bayes Statistical Model Evaluation")
print(metrics.confusion_matrix(test_Y, predSWNB, labels=["Real","Fake"]))
print(metrics.classification_report(test_Y, predSWNB, labels=["Real","Fake"]))

print("\n\n")

print("Removed Stopwords: Boolean Naive Bayes Statistical Model Evaluation")
print(metrics.confusion_matrix(test_Y, predSWBNB, labels=["Real","Fake"]))
print(metrics.classification_report(test_Y, predSWBNB, labels=["Real","Fake"]))

Removed Stopwords: Simple Naive Bayes Statistical Model Evaluation
[[150   0]
 [ 90   0]]
              precision    recall  f1-score   support

        Real       0.62      1.00      0.77       150
        Fake       0.00      0.00      0.00        90

    accuracy                           0.62       240
   macro avg       0.31      0.50      0.38       240
weighted avg       0.39      0.62      0.48       240




Removed Stopwords: Boolean Naive Bayes Statistical Model Evaluation
[[150   0]
 [ 90   0]]
              precision    recall  f1-score   support

        Real       0.62      1.00      0.77       150
        Fake       0.00      0.00      0.00        90

    accuracy                           0.62       240
   macro avg       0.31      0.50      0.38       240
weighted avg       0.39      0.62      0.48       240



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Synopsis

After a very long and hectic implementation of 4 type of model, I come to conclusion which is stated below
  

1.   Boolean Naive bayes and simple Naive Bayes  perform similar in term of evaluations. They Accuracy Score by Both these Algorithms has 62 percent.

2.   Removing stopwords cause no betterment in the acuuracy